# Use PyTorch to predict handwritten digits
<table style="border: none" align="left">
   <tr style="border: none">
       <td style="border: none"><img src="https://github.com/IBM/pytorch-on-watson-studio/raw/master/doc/source/images/pytorch-pattern-header.jpg" width="600" alt="Icon"></td>
   </tr>
</table>

This notebook contains steps and code to demonstrate Deep Learning model
training in the
<a href="https://www.ibm.com/cloud/machine-learning">Watson Machine Learning</a>
service.

<a href="https://pytorch.org/" target="_blank" rel="noopener no
referrer">PyTorch</a> is a relatively new deep learning framework. Yet, it has
begun to gain adoption especially among researchers and data scientists. The
strength of PyTorch is its support of dynamic computational graph while most deep
learning frameworks are based on static computational graph. In addition, its strong
NumPy like GPU accelerated tensor computation has allowed Python developers to easily
learn and build deep learning networks for GPUs and CPUs alike.

Some familiarity with Python is helpful. This notebook uses Python 3 and
<a href="https://dataplatform.cloud.ibm.com/docs/content/analyze-data/environments-parent.html" target="_blank" rel="noopener no referrer">Watson Studio</a> to configure and initiate training of a PyTorch base
workload using Watson Machine Learning service.


## Learning goals

In this notebook, you will learn how to:

-  Work with Watson Machine Learning to train Deep Learning models
-  Use PyTorch features, tools and libraries  
-  Save trained models in the Watson Machine Learning repository

## Contents

1.	[Set up](#setup)
2.	[Create the training definitions](#model)
3.  [Train the model](#train)
4.	[Work with the trained models](#work)
5.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.bluemix.net/catalog/services/machine-learning" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance is <a href="https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener no referrer">here</a>).
-  Create a <a href="https://console.bluemix.net/catalog/services/cloud-object-storage" target="_blank" rel="noopener no referrer">Cloud Object Storage (COS)</a> instance (a lite plan is offered and information about how to order storage is <a href="https://console.bluemix.net/docs/services/cloud-object-storage/basics/order-storage.html#order-storage" target="_blank" rel="noopener no referrer">here</a>). <br/>**Note: When using Watson Studio, you already have a COS instance associated with the project you are running the notebook in.**
- Create new credentials with HMAC: 
    - Go to your COS dashboard (see Tip).
    - In the **Service credentials** tab, click **New Credential+**.
    - In the **Add Inline Configuration Parameters(Optional):** box, add {"HMAC":true}
    - Click **Add**. (For more information, see <a href="https://console.bluemix.net/docs/services/cloud-object-storage/hmac/credentials.html#using-hmac-credentials" target="_blank" rel="noopener no referrer">HMAC</a>.)

    This configuration parameter adds the following section to the instance credentials, (for use later in this notebook):
    ```
      "cos_hmac_keys": {
            "access_key_id": "-------",
            "secret_access_key": "-------"
       }
    ```
 
**Tip:** follow the steps below to access your COS instance dashboard. From the Watson Studio dashboard:
- Click the **Services** tab on the top of the page
- Click the **Data Services** tab
- Select and click your target object storage (COS)

### 1.1 Work with Cloud Object Storage  (COS)

Install the boto library. This library allows Python developers to manage Cloud Object Storage (COS).

**Tip:** If `ibm_boto3` is not preinstalled in your environment, run the following command to install it: 

In [1]:
# Run the command if ibm_boto3 is not installed.
# !pip install ibm-cos-sdk

In [2]:
# Install the boto library.
import ibm_boto3
from ibm_botocore.client import Config

**Replace** the information in the following cell with your COS credentials. 

You can find these credentials in your COS instance dashboard under the **Service credentials** tab.

**Note:** the HMAC key, described in [set up the environment](#setup) is included in these credentials.

`
cos_credentials = {
  "apikey": "-------",
  "cos_hmac_keys": {
    "access_key_id": "------",
    "secret_access_key": "------"
  },
  "endpoints": "https://cos-service.bluemix.net/endpoints",
  "iam_apikey_description": "------",
  "iam_apikey_name": "------",
  "iam_role_crn": "------",
  "iam_serviceid_crn": "------",
  "resource_instance_id": "-------"
}
`

In [3]:
# @hidden_cell
cos_credentials = {
  # Credentials deleted
}

Define the endpoint.

To do this, go to the **Endpoint** tab in the COS instance's dashboard to get the endpoint information, then enter it in the cell below:

In [4]:
# Define endpoint information.
service_endpoint = 'https://s3-api.us-geo.objectstorage.softlayer.net'

You also need the IBM Cloud authorization endpoint to be able to create COS resource object.

In [5]:
# Define the authorization endpoint.
auth_endpoint = 'https://iam.bluemix.net/oidc/token'

Create a Boto resource to be able to write data to COS.

In [6]:
# Create a COS resource.
cos = ibm_boto3.resource('s3',
                         ibm_api_key_id=cos_credentials['apikey'],
                         ibm_service_instance_id=cos_credentials['resource_instance_id'],
                         ibm_auth_endpoint=auth_endpoint,
                         config=Config(signature_version='oauth'),
                         endpoint_url=service_endpoint)

Create two buckets, which you will use to store training data and training results.

**Note:** The bucket names must be unique.

In [7]:
from uuid import uuid4

bucket_uid = str(uuid4())
buckets = ['training-mnist-data-' + bucket_uid, 'training-mnist-results-' + bucket_uid]

for bucket in buckets:
    if not cos.Bucket(bucket) in cos.buckets.all():
        print('Creating bucket "{}"...'.format(bucket))
        try:
            cos.create_bucket(Bucket=bucket)
        except ibm_boto3.exceptions.ibm_botocore.client.ClientError as e:
            print('Error: {}.'.format(e.response['Error']['Message']))

Creating bucket "training-mnist-data-65764683-d6c9-41ef-8b00-7542f4dc789c"...
Creating bucket "training-mnist-results-65764683-d6c9-41ef-8b00-7542f4dc789c"...


Now you should have 2 buckets.

In [8]:
# Display a list of created buckets.
print(list(cos.buckets.all()))

[s3.Bucket(name='project1128-donotdelete-pr-tgpezbwx13exei'), s3.Bucket(name='project1130-donotdelete-pr-nxsinhfnbatswh'), s3.Bucket(name='test13-donotdelete-pr-snifuyqryecjf7'), s3.Bucket(name='training-mnist-data-65764683-d6c9-41ef-8b00-7542f4dc789c'), s3.Bucket(name='training-mnist-results-65764683-d6c9-41ef-8b00-7542f4dc789c'), s3.Bucket(name='wwmlpytorch-donotdelete-pr-jexsjlamd39bhr')]


### 1.2 Download the training data and upload it to the COS buckets

**PyTorch Tools & Libraries**

An active community of researchers and developers have built a rich
ecosystem of tools and libraries for extending PyTorch and supporting
development in areas from computer vision to reinforcement learning.

PyTorch's <a href="https://github.com/pytorch/vision" target="_blank"
rel="noopener no referrer">torchvision</a> is one of those packages.
`torchvision` consists of popular datasets, model architectures, and common
image transformations for computer vision.

This tutorial will use `torchvision's MNIST dataset` package to download
and process the training data. The processed data files will be uploaded to
the `training-data-mnist` bucket.

**Tip:** If PyTorch or `torchvision` is not preinstalled in your environment, run the
following command to install it: 

In [9]:
#Install PyTorch
!pip install http://download.pytorch.org/whl/cpu/torch-0.4.1-cp35-cp35m-linux_x86_64.whl
#Install torchvision
!pip install torchvision

    100% |████████████████████████████████| 91.1MB 97.1MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 8.2MB/s eta 0:00:01
Requirement not upgraded as not directly required: torch in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from torchvision)
Requirement not upgraded as not directly required: pillow>=4.1.1 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from torchvision)
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from torchvision)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from torchvision)
Requirement not upgraded as not directly required: olefile in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from pillow>=4.1.1->torchvision)


The following code will download and process the MNIST training and test data. 

In [10]:
import torch
from torchvision import datasets, transforms

data_dir = './data'

datasets.MNIST(data_dir, train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))

Processing...
Done!


Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: ./data
    Transforms (if any): Compose(
                             ToTensor()
                             Normalize(mean=(0.1307,), std=(0.3081,))
                         )
    Target Transforms (if any): None

The code in the next cell uploads the processed files to your COS.

In [11]:
import glob
import os

files_search = os.path.join(data_dir, "processed", "*")
files = glob.glob(files_search)

bucket_obj = cos.Bucket(buckets[0])

for file in files:
    filename = file.split('/')[-1]
    filename = os.path.join("processed", filename)
    print('Uploading data {}...'.format(filename))
    bucket_obj.upload_file(file, filename )
    print('{} is uploaded.'.format(filename))
print("Done")    

Uploading data processed/training.pt...
processed/training.pt is uploaded.
Uploading data processed/test.pt...
processed/test.pt is uploaded.
Done


Have a look at the list of the created buckets and their contents.

In [12]:
for bucket_name in buckets:
    print(bucket_name)
    bucket_obj = cos.Bucket(bucket_name)
    for obj in bucket_obj.objects.all():
        print("  File: {}, {:4.2f}kB".format(obj.key, obj.size/1024))

training-mnist-data-65764683-d6c9-41ef-8b00-7542f4dc789c
  File: processed/test.pt, 7734.80kB
  File: processed/training.pt, 46406.67kB
training-mnist-results-65764683-d6c9-41ef-8b00-7542f4dc789c


You are done with COS, and you are ready to train your model!

### 1.3. Work with the WML service instance

Import the libraries you need to work with your WML instance.

**Hint:** You may also need to install `wget` using the following command `!pip install wget`

In [13]:
!pip install wget

  Running setup.py bdist_wheel for wget ... done
  Stored in directory: /home/dsxuser/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [14]:
import urllib3, requests, json, base64, time, os, wget

Authenticate to the Watson Machine Learning (WML) service on IBM Cloud.

**Tip**: Authentication information (your credentials) can be found in the <a href="https://console.bluemix.net/docs/services/service_credentials.html#service_credentials" target="_blank" rel="noopener noreferrer">Service credentials</a> tab of the service instance that you created on IBM Cloud. 
If there are no credentials listed for your instance in **Service credentials**, click **New credential (+)** and enter the information required to generate new authentication information. 

**Action**: Enter your WML service instance credentials here.

`
wml_credentials = {
  "apikey": "------",
  "iam_apikey_description": "------:",
  "iam_apikey_name": "------",
  "iam_role_crn": "-------",
  "iam_serviceid_crn": "-------",
  "instance_id": "-------",
  "password": "------",
  "url": "------",
  "username": "-------"
}
`

In [15]:
# @hidden_cell
wml_credentials = {
  # Credentials deleted
}

#### Import the `watson-machine-learning-client` and authenticate to the service instance.

**Tip:** If `watson-machine-learning-client` is not preinstalled in your environment, run the following command to install it: 

In [16]:
# !pip install watson-machine-learning-client

In [17]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


**Note:** A deprecation warning is returned from scikit-learn package that does not impact watson machine learning client functionalities.

In [18]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [19]:
# Display the client version number.
print(client.version)

1.0.345


**Note:** `watson-machine-learning-client` documentation can be found <a href="http://wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener noreferrer">here</a>.

<a id="model"></a>
## 2. Create the training definitions

In this section you:

- [2.1 Prepare the training definition metadata](#prep)
- [2.2 Get the sample model definition content files from Git](#get)
- [2.3 Store the training definition in the WML repository](#store)



### 2.1 Prepare the training definition metadata<a id="prep"></a>

Prepare the training definition metadata. The main program will be called with
enviroment variables `$DATA_DIR` and `$RESULT_DIR` as the inputs for the
`--data-dir` and `--result-dir` options.

**Tip:** You may want to change the number of epoch to run with a larger epoch number.

In [20]:
model_definition_metadata = {
            client.repository.DefinitionMetaNames.NAME: "My definition name",
            client.repository.DefinitionMetaNames.DESCRIPTION: "My description",
            client.repository.DefinitionMetaNames.AUTHOR_NAME: "John Smith",
            client.repository.DefinitionMetaNames.FRAMEWORK_NAME: "pytorch",
            client.repository.DefinitionMetaNames.FRAMEWORK_VERSION: "0.4",
            client.repository.DefinitionMetaNames.RUNTIME_NAME: "python",
            client.repository.DefinitionMetaNames.RUNTIME_VERSION: "3.5",
            client.repository.DefinitionMetaNames.EXECUTION_COMMAND: "python3 main.py --epochs 1 --data-dir $DATA_DIR --result-dir $RESULT_DIR"
            }

### 2.2 Get the sample model definition content file from GitHub <a id="get"></a>

The sample model used here is the <a href="https://github.com/pytorch/examples/tree/master/mnist">MNIST model</a> from the official PyTorch examples repository. 

In [21]:
filename='pytorch-mnist.zip'

if not os.path.isfile(filename):
    filename = wget.download('https://github.com/IBM/pytorch-on-watson-studio/raw/master/data/code/pytorch-mnist.zip')
    print(filename, "was downloaded")
else:
    print(filename, "was downloaded previously.")


pytorch-mnist.zip was downloaded


You can verify the size of the model definition file by running the following command.
```
ls -o
```

### 2.3 Store the training definition in the WML repository<a id="store"></a>

In [22]:
definition_details = client.repository.store_definition(filename, model_definition_metadata)

In [23]:
definition_uid = client.repository.get_definition_uid(definition_details)

In [24]:
# Display the training definition uid.
print(definition_uid)

65544ef2-4dfb-462e-ada3-d49763358085


## 3. Train the model<a id="train"></a>

In this section, learn how to:
- [3.1 Enter training configuration metadata](#meta)
- [3.2 Train the model in the background](#backg)
- [3.3 Monitor the training log](#log)
- [3.4 Cancel the training run](#cancel)


### 3.1 Enter training configuration metadata<a id="meta"></a>

- `TRAINING_DATA_REFERENCE` - references the uploaded training data.
- `TRAINING_RESULTS_REFERENCE` - location where trained model will be saved.

**Note** Your COS credentials are referenced in this code.


In [25]:
# Configure the training metadata for the TRAINING_DATA_REFERENCE and TRAINING_RESULTS_REFERENCE.
training_configuration_metadata = {
            client.training.ConfigurationMetaNames.NAME: "Hand-written Digit Recognition", 
            client.training.ConfigurationMetaNames.AUTHOR_NAME: "John Smith",              
            client.training.ConfigurationMetaNames.DESCRIPTION: "Hand-written Digit Recognition training",
            client.training.ConfigurationMetaNames.COMPUTE_CONFIGURATION: {"name": "k80"},
            client.training.ConfigurationMetaNames.TRAINING_DATA_REFERENCE: {
                    "connection": {
                        "endpoint_url": service_endpoint,
                        "access_key_id": cos_credentials['cos_hmac_keys']['access_key_id'],
                        "secret_access_key": cos_credentials['cos_hmac_keys']['secret_access_key']
                    },
                    "source": {
                        "bucket": buckets[0],
                    },
                    "type": "s3"
                },
            client.training.ConfigurationMetaNames.TRAINING_RESULTS_REFERENCE: {
                "connection": {
                    "endpoint_url": service_endpoint,
                    "access_key_id": cos_credentials['cos_hmac_keys']['access_key_id'],
                    "secret_access_key": cos_credentials['cos_hmac_keys']['secret_access_key']
                },
                "target": {
                    "bucket": buckets[1],
                },
                "type": "s3"
            },
        }

### 3.2 Train the model in the background<a id="backg"></a>

To run the training in the **background**, set the optional parameter `asynchronous=True` (or remove it). In this case the parameter has been removed. 

**Note:** To run the training in **active** mode, set `asynchronous=False`.

In [26]:
training_run_details = client.training.run(definition_uid, training_configuration_metadata)
# print(json.dumps(training_run_details, indent=2))

In [27]:
training_run_guid_async = client.training.get_run_uid(training_run_details)
print("training_run_guid_async=",training_run_guid_async)

training_run_guid_async= model-dife4hxz


Check the status of the training run by calling the method the next cell:

In [28]:
# Get training run status.
status = client.training.get_status(training_run_guid_async)
print(json.dumps(status, indent=2))

{
  "metrics": [],
  "submitted_at": "2018-11-30T17:29:12Z",
  "message": "training-NrCjxLLmg: Submitted",
  "current_at": "2018-11-30T17:29:15Z",
  "state": "pending"
}


### 3.3  Monitor the training log<a id="log"></a>

Run the cell below to monitor the training log. 

In [29]:
client.training.monitor_logs(training_run_guid_async)



####################################################

Log monitor started for training run: model-dife4hxz

####################################################


training-NrCjxLLmg: Training with training/test data at:

training-NrCjxLLmg:   DATA_DIR: /mnt/data/training-mnist-data-65764683-d6c9-41ef-8b00-7542f4dc789c

training-NrCjxLLmg:   MODEL_DIR: /job/model-code

training-NrCjxLLmg:   TRAINING_COMMAND: python3 main.py --epochs 1 --data-dir $DATA_DIR --result-dir $RESULT_DIR

training-NrCjxLLmg:   LOG_DIR: /job/logs

training-NrCjxLLmg:   RESULT_DIR: /mnt/results/training-mnist-results-65764683-d6c9-41ef-8b00-7542f4dc789c/training-NrCjxLLmg

training-NrCjxLLmg:   NUM_LEARNERS: 1

training-NrCjxLLmg: Fri Nov 30 17:29:55 UTC 2018: Running PyTorch job

training-NrCjxLLmg: Train Epoch: 1 [0/60000 (0%)]	Loss: 2.373651

training-NrCjxLLmg: Train Epoch: 1 [640/60000 (1%)]	Loss: 2.310517

training-NrCjxLLmg: Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.281828

training-NrCjxLLmg: Train Epoch

After the training is complete, get the training GUID.

In [30]:
training_details = client.training.get_details(training_run_guid_async)
training_guid = training_details["entity"]["training_results_reference"]["location"]["model_location"]
print("Training GUID is:", training_guid)

Training GUID is: training-NrCjxLLmg


### 3.4 Cancel the training run<a id="cancel"></a>

You can cancel the training run by calling the method below.
```
client.training.cancel(training_run_guid_async)
```

<a id="work"></a>
## 4. Work with the trained models

In this sample workload, the trained model is saved as a file named
`saved_models.pth` in the result bucket.
The following code will fetch the model file from the bucket.

**Tip:** Make sure that the training run is completed by checking it's
status as shown earlier.

In [31]:
# buckets[1] is the bucket to save the result data as defined above
bucket_obj = cos.Bucket(buckets[1])
# model file name as defined in the code 
saved_model_filename = "saved_models.pth"
source_file = os.path.join(training_guid, saved_model_filename)
bucket_obj.download_file(source_file,saved_model_filename)

Copy the definition of the neural network as it is defined in the sample workload. 

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

Instantiate and load previously trained model parameters.

In [33]:
mnist_model = Net()
mnist_model.load_state_dict(torch.load(saved_model_filename, map_location='cpu'))

Download sample image files.

In [34]:
import os
import wget

images =[]
for i in range(1,10):
    filename = "img_"+str(i)+".jpg"
    images.append(filename)
    if not os.path.isfile(filename):
        path = "https://github.com/IBM/pytorch-on-watson-studio/raw/master/data/images/"+filename
        wget.download(path)

print(images)

['img_1.jpg', 'img_2.jpg', 'img_3.jpg', 'img_4.jpg', 'img_5.jpg', 'img_6.jpg', 'img_7.jpg', 'img_8.jpg', 'img_9.jpg']


Using the trained model to predict the digits in the sample image files.

In [35]:
import numpy as np
from IPython.display import display
from PIL import Image

**Python-First**

PyTorch is not a Python binding into a monolithic C++ framework.
It’s built to be deeply integrated into Python so it can
be used with popular Python libraries.

The code below shows how to convert a NumPy array to
a PyTorch tensor using `torch.from_numpy`.

In [36]:
digits = [i for i in range(10)]

mnist_model.eval()
for i, filename in enumerate(images):
    img = Image.open(filename).resize((28, 28)).convert('L')
    display(img)
    data = torch.from_numpy(np.asarray(img, dtype=np.float32)[np.newaxis, np.newaxis, :, :])
    output = mnist_model(data)
    # get the index of the max log-probability
    prediction = output.max(1, keepdim=True)[1]
    print("Prediction for image number", i+1, "is:",  digits[prediction[0,0]])

Prediction for image number 1 is: 2


Prediction for image number 2 is: 0


Prediction for image number 3 is: 9


Prediction for image number 4 is: 9


Prediction for image number 5 is: 3


Prediction for image number 6 is: 7


Prediction for image number 7 is: 0


Prediction for image number 8 is: 3


Prediction for image number 9 is: 0


**Native ONNX Support**

PyTorch includes native <a href="http://onnx.ai/">Open Neural Network Exchange (ONNX)</a> support. 

The following code will export models in the standard ONNX
format so that the models can be consumed
by ONNX-compatible platforms, runtimes, visualizers, and more.

PyTorch exports the model by running the model through the training path once
and then save the traced model to a file using ONNX format.

**Tip:** You can test the exported ONNX format model by importing and
running it in an ONNX-comptible famework. See
<a href="https://github.com/onnx/tutorials/tree/master/tutorials/PytorchTensorflowMnist.ipynb">ONNX tutorials</a>
for more information. 


In [37]:
# Export the trained model to ONNX
# one black and white 28 x 28 picture will be used as the input to the model
dummy_input = torch.randn(1, 1, 28, 28) 

onnx_model_filename = "mnist.onnx"
torch.onnx.export(mnist_model, dummy_input,onnx_model_filename)

Save the ONNX model file to the result bucket.

In [38]:
# buckets[1] is bucket to save the result data as defined above
bucket_obj = cos.Bucket(buckets[1])
# model file name as defined in the code 
bucket_obj.upload_file(onnx_model_filename,onnx_model_filename)

You are done and can delete the training run in WML by calling the method below.

```
client.training.delete(training_run_uid_async)
```

<a id="summary"></a>
## 5. Summary and next steps     

You successfully completed this notebook! 
 
You learned how to use `watson-machine-learning-client` to train PyTorch models. 
 
Check out our <a href="https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener noreferrer">Online Documentation</a> for a <a href="https://dataplatform.ibm.com/docs/content/analyze-data/ml-python-mnist-tutorial.html" target="_blank" rel="noopener noreferrer">tutorial</a> and more samples, documentation, how-tos, and blog posts. 

### Citations
Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner. "Gradient-based learning applied to document recognition." Proceedings of the IEEE, 86(11):2278-2324, November 1998.

### References
1. <a href="https://pytorch.org/">PyTorch</a>.
2. <a href="https://github.com/pytorch/examples/tree/master/mnist">MNIST model</a> from the official PyTorch examples repository.
3. <a href="https://dataplatform.cloud.ibm.com/analytics/notebooks/v2/3bd3efb8-833d-460f-b07b-fee51dd0f1af/view?access_token=6bd0ff8d807861d09e0dab0cad28ce9685711078f612fcd92bb8cf8535d089c1">Use TensorFlow to predict handwritten digits</a>

### Authors

**Lucasz Cmielowski**, PhD, is a Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increase the clients' ability to turn data into actionable knowledge.

**Catherine Diep** is a Solutions Architect and Performance Engineer of the Cognitive OpenTech group at IBM Silicon Valley Lab. Her current projects include deep learning related workloads that use open source frameworks and APIs such as PyTorch, TensorFlow, Keras, etc.

**Simeon Monov** is a Senior Software Developer and Performance Engineer for the Cognitive OpenTech group at IBM. He is currently working in data science and machine learning related projects.

Copyright © 2017, 2018 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>